# Caipora Project

__Objetivo__




 
__Data Source__

https://queimadas.dgi.inpe.br/queimadas/portal

https://ipsamazonia.org.br/

https://openaq.org/


__Data characteristics__

- Time Series;
- Geographic coordinates – Latitude/Longitude;
- Satellite Name

In [1]:
import os
import math
import unicodedata

import glob

import datetime
import gmaps

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

## Prerequisite

In [2]:
WORKDIR = os.path.abspath(os.getcwd())

## Get the data

### Hotspot data

__List files used in the analysis__

In [3]:
path = ''.join([WORKDIR, "/data/hotspot/**/*"]) 
hotspot_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [4]:
hotspot_df = pd.concat(map(pd.read_csv, hotspot_files))
hotspot_df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,CURURUPU,Amazonia,0.0,0.8,0.0,-1.87136,-44.78587,NaN
1,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,MARACACUME,Amazonia,0.0,0.1,0.1,-1.82566,-45.88670,NaN
2,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,BURITICUPU,Amazonia,0.0,1.1,0.1,-4.57874,-46.38660,NaN
3,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,PAULO RAMOS,Amazonia,0.0,1.4,0.1,-4.59554,-45.66039,NaN
4,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,ARAME,Amazonia,0.0,0.4,0.3,-5.21960,-46.12886,NaN


### Amazon SPI data

__List files used in the analysis__

In [5]:
path = ''.join([WORKDIR, "/data/spi/amazonia/detailed"])
spi_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [6]:
spi_df = pd.concat(map(pd.read_csv, spi_files))
spi_df.head()

,Unnamed: 0,Código IBGE,Município,Estado,IPS Amazônia 2014,Ranking IPS,Necessidades Humanas Básicas,Fundamentos para o Bem-Estar,Oportunidades,Nutrição e cuidados médicos básicos,...,"Desmatamento recente 2019, 2019, 2020 (% área total do município)",Focos de calor 2020 (nª de focos/1.000 habitantes),Diversidade Partidária 2020 (% vereadores eleitos partidos diferentes),Transporte Público 2020 (nº de ônibus e micro-ônibus/1.000 habitantes),"Acesso à cultura, esporte e lazer 2018 (Categórica 1-10)",Gravidez na infância e adolescência 2019 (% de filhos de mães com até 19 anos),Trabalho Infantil 2019 (nº de famílias com ao menos 1 membro em trabalho infantil/1.000 famílias),Vulnerabilidade familiar 2019 (% de filhos de mães solteiras),Empregos ensino superior 2019 (% de empregos em relação ao total),Mulheres com empregos ensino superior 2019 (% de empregos em relação ao total)
0,0,1100015.0,Alta Floresta D'Oeste,RO,56.594170,197.0,60.195511,57.369340,52.217659,95.154708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1100023.0,Ariquemes,RO,55.728511,264.0,59.059533,59.304304,48.821695,93.920755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1100031.0,Cabixi,RO,58.915724,90.0,74.178132,54.787265,47.781775,88.092664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1100049.0,Cacoal,RO,61.717654,28.0,72.691093,60.613765,51.848105,91.956107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1100056.0,Cerejeiras,RO,54.593926,361.0,62.606094,62.512401,38.663282,87.810163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explore the data

### Describe the data I

__Hotspot__

In [7]:
hotspot_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
diasemchuva,11060285.0,7.623764,70.880709,-999.00000,1.00000,4.00000,12.000,120.00000
precipitacao,11060285.0,1.034430,3.854016,0.00000,0.00000,0.00000,0.300,203.70000
riscofogo,11060285.0,-5.310438,77.318383,-999.00000,0.40000,0.90000,1.000,1.00000
latitude,12651992.0,-7.019591,3.980039,-16.29000,-9.85041,-7.71500,-3.879,5.23000
longitude,12651992.0,-56.207436,6.320051,-73.93146,-61.00200,-55.22246,-51.210,-43.64537
frp,3390008.0,19.308732,59.705760,-3.70000,3.00000,7.20000,16.800,9722.60000


__Amazon SPI__

In [8]:
spi_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,2319.0,3.860000e+02,2.231938e+02,0.000000e+00,1.930000e+02,3.860000e+02,5.790000e+02,7.720000e+02
Código IBGE,2316.0,2.288414e+06,1.365403e+06,1.100015e+06,1.502798e+06,1.713254e+06,2.109278e+06,5.108956e+06
IPS Amazônia 2014,772.0,5.445173e+01,3.733595e+00,4.509119e+01,5.177761e+01,5.425550e+01,5.665038e+01,7.171051e+01
Ranking IPS,2316.0,3.865000e+02,2.229051e+02,1.000000e+00,1.937500e+02,3.865000e+02,5.792500e+02,7.720000e+02
Necessidades Humanas Básicas,2319.0,6.437410e+01,6.825254e+00,4.337849e+01,5.969696e+01,6.432076e+01,6.887852e+01,8.712833e+01
...,...,...,...,...,...,...,...,...
Gravidez na infância e adolescência 2019 (% de filhos de mães com até 19 anos),772.0,2.384973e+01,6.295921e+00,5.172414e+00,1.955196e+01,2.412254e+01,2.815534e+01,4.545455e+01
Trabalho Infantil 2019 (nº de famílias com ao menos 1 membro em trabalho infantil/1.000 famílias),772.0,3.170346e+01,6.065901e+01,0.000000e+00,2.748239e+00,9.631206e+00,3.342557e+01,4.740000e+02
Vulnerabilidade familiar 2019 (% de filhos de mães solteiras),772.0,4.370381e+01,2.095850e+01,5.000000e+00,2.629215e+01,3.899752e+01,5.949844e+01,9.534161e+01
Empregos ensino superior 2019 (% de empregos em relação ao total),772.0,1.828965e+01,1.281655e+01,4.964422e+00,1.129523e+01,1.595505e+01,2.173583e+01,1.479940e+02


### Get information about data

__Hotspot__

In [9]:
hotspot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12651992 entries, 0 to 318737
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   datahora      object 
 1   satelite      object 
 2   pais          object 
 3   estado        object 
 4   municipio     object 
 5   bioma         object 
 6   diasemchuva   float64
 7   precipitacao  float64
 8   riscofogo     float64
 9   latitude      float64
 10  longitude     float64
 11  frp           float64
dtypes: float64(6), object(6)
memory usage: 1.2+ GB


__Amazon SPI__

In [10]:
spi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2319 entries, 0 to 772
Columns: 160 entries, Unnamed: 0 to Mulheres com empregos ensino superior 2019 (% de empregos em relação ao total)
dtypes: float64(157), int64(1), object(2)
memory usage: 2.8+ MB


### Percentage of missing data

In [11]:
def get_pct_missing_data(dataset):
    """
    Get Percentage of missing data
    
    Attributes
    ----------
    dataset : Pandas DataFrame
    """
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum() / dataset.isnull().count() * 100
    percent = (round(percent, 4)).sort_values(ascending=False)

    missing_data = pd.concat([total, percent], keys=["Total", '%'], axis=1)

    return missing_data

__Hotspot__

In [12]:
get_pct_missing_data(hotspot_df)

,Total,%
frp,9261984,73.2057
diasemchuva,1591707,12.5807
precipitacao,1591707,12.5807
riscofogo,1591707,12.5807
datahora,0,0.0000
satelite,0,0.0000
pais,0,0.0000
estado,0,0.0000
municipio,0,0.0000
bioma,0,0.0000


__Amazon SPI__

In [13]:
get_pct_missing_data(spi_df)

,Total,%
Moradias com iluminação adequada 2018 (% de domicilios),1547,66.7098
Vulnerabilidade familiar 2017 (% de filhos de mães solteiras),1547,66.7098
Desmatamento acumulado 2017 (% área total do município),1547,66.7098
"Desmatamento recente 2017, 2017, 2018 (% área total do município)",1547,66.7098
Focos de calor 2018 (nª de focos/1.000 habitantes),1547,66.7098
...,...,...
Segurança pessoal,0,0.0000
Acesso ao conhecimento básico,0,0.0000
Acesso à informação e comunicação,0,0.0000
Saúde e bem-estar,0,0.0000


## Prepare the data

### Parse datehour of string to date

__Hotspot__

In [14]:
hotspot_df['datahora'] = pd.to_datetime(hotspot_df['datahora'], format='%Y/%m/%d %H:%M:%S')

### Add Year and Month columns

__Hotspot__

In [15]:
hotspot_df["ano"] = pd.DatetimeIndex(hotspot_df['datahora']).year
hotspot_df["mes"] = pd.DatetimeIndex(hotspot_df['datahora']).month
hotspot_df["dia"] = pd.DatetimeIndex(hotspot_df['datahora']).day

### Fix or remove outliers (optional)

__Hotspot__

In [16]:
hotspot_df.loc[hotspot_df["riscofogo"] < 0, "riscofogo"] = 0
hotspot_df.loc[hotspot_df["diasemchuva"] < 0, "diasemchuva"] = 0

__Amazon SPI__

In [17]:
spi_df.dropna(subset=["cibge", "municipio", "estado", "ips"], inplace=True)

KeyError: ['cibge', 'municipio', 'estado', 'ips']

In [ ]:
def purge_spec_chars(word):
    """
    Remove all special characters in the word.
    
    Attributes
    ----------
    word : str
    
    Returns
    -------
    word_without_spec_chars: str
    """
    nfkd_form = unicodedata.normalize("NFKD", word)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
spi_df.municipio = spi_df.municipio.apply(purge_spec_chars)

### Fill in missing values (e.g., with zero, mean, median...) or drop their rows (or columns)

__Hotspot__

In [ ]:
hotspot_df['frp'].fillna(value=0, inplace=True)
hotspot_df['riscofogo'].fillna(value=0, inplace=True)
hotspot_df['diasemchuva'].fillna(value=0, inplace=True)
hotspot_df['precipitacao'].fillna(value=0, inplace=True)

### Drop columns not used

__Hotspot__

In [ ]:
hotspot_df = hotspot_df.drop(columns=["satelite", "pais", "bioma", "latitude", "longitude"])
hotspot_df.head()

### Describe the data II

__Hotspot__

In [ ]:
hotspot_df.describe().transpose()

In [ ]:
get_pct_missing_data(hotspot_df)

__Amazon SPI__

In [ ]:
spi_df.describe().transpose()

In [ ]:
get_pct_missing_data(spi_df)